In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [2]:
import visdom
vis = visdom.Visdom()
vis.close(env="main")

Setting up a new session...


''

In [3]:
#define loss tracker
def loss_tracker(loss_plot, loss_value, num):
    vis.line(Y=loss_value, X=num, win=loss_plot, update='append')

In [4]:
#set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [5]:
#cifar10 data settings
import ssl
ssl._create_default_https_context = ssl._create_unverified_context #ssl 문제해결용 임시 코드
trans = transforms.Compose(
        [transforms.ToTensor(), 
        transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))])
trainset = torchvision.datasets.CIFAR10(root='./cifar10', train=True,
                                       download=True, transform=trans)
trainloader = torch.utils.data.DataLoader(dataset=trainset, batch_size=512,
                                         shuffle=True, num_workers=4)
testset = torchvision.datasets.CIFAR10(root='./cifar10', train=False,
                                      download=False, transform=trans)
testloader = torch.utils.data.DataLoader(dataset=testset, batch_size=4,
                                        shuffle=True, num_workers=4)
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified


In [6]:
import torchvision.models.vgg as vgg

In [7]:
cfg = [32,32,'M',64,64,128,128,128,'M',256,256,256,512,512,512,'M'] #13+3 = vgg16

In [8]:
#Class VGG
class VGG(nn.Module):
    def __init__(
        self, features: nn.Module, num_classes: int = 1000, init_weights: bool = True, dropout: float = 0.5
    ) -> None:
        super().__init__()
        self.features = features
        self.classifier = nn.Sequential(
            nn.Linear(512 * 4 * 4, 4096),
            nn.ReLU(True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, num_classes),
        )
        if init_weights:
            for m in self.modules():
                if isinstance(m, nn.Conv2d):
                    nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
                elif isinstance(m, nn.BatchNorm2d):
                    nn.init.constant_(m.weight, 1)
                    nn.init.constant_(m.bias, 0)
                elif isinstance(m, nn.Linear):
                    nn.init.normal_(m.weight, 0, 0.01)
                    nn.init.constant_(m.bias, 0)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [9]:
#model
vgg16 = VGG(vgg.make_layers(cfg), 10, True).to(device)

In [10]:
#model의 output 형태가 10 class로 나오는지 확인
a = torch.Tensor(1,3,32,32).to(device)
out = vgg16(a)
print(out)

tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]],
       grad_fn=<AddmmBackward0>)


In [11]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(vgg16.parameters(), lr = 0.005, momentum=0.9)
#scheduler 사용(lr 줄이면서 학습)
lr_sche = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.9)

In [12]:
#make plot
loss_plot = vis.line(Y=torch.Tensor(1).zero_(), opts=dict(title='loss_tracker', legend=['loss'], showlegend=True))

In [ ]:
#training
training_epochs = 50

for epoch in range(training_epochs):
    running_loss = 0.0
    lr_sche.step()
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        prediction = vgg16(inputs)
        cost = criterion(prediction, labels)
        cost.backward()
        optimizer.step()
        
        running_loss += cost.item()
        if i%30 ==29 :
            loss_tracker(loss_plot, torch.Tensor([running_loss/30]), torch.Tensor([i + epoch*len(trainloader)]))
            print('[%d,%d] loss: %.3f'%(epoch+1, i+1, running_loss/30))
            running_loss = 0.0

C:\anaconda3\envs\tensorflow\lib\site-packages\torch\optim\lr_scheduler.py:129: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


[1,30] loss: 2.302
[1,60] loss: 2.296
[1,90] loss: 2.277
[2,30] loss: 2.163


In [ ]:
#test(imshow)
dataiter = iter(testloader)
images, labels = dataiter.next()

imshow(torchvision.utils.make_grid(images))
print('GroundTruth:', ' '.join('%5s'%classes[labels[j]] for j in range(4)))

outputs = vgg16(images.to(device))
_, predicted = torch.max(outputs, 1)
print('Predicted:', ' '.join('%5s'%classes[predicted[j]] for j in range(4)))

In [ ]:
#test(accuracy)
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = vgg16(images)
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy of the network on the 10000 test images: %d %%'%(100*correct/total))